# Utilisation API Pôle emploi
Source : https://github.com/etiennekintzler/api-offres-emploi

In [172]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import HTTPError

import datetime
import time
import re
import pandas as pd 

In [173]:
client_id="PAR_textmining_6361ebc3d9de749cdfb72158572fa975a0ba30e97e4e570957f187132bb2b361"
client_secret="d77c395663a600cb89716cd8a6c02ab9b2d6920fbdc520791bcff93578b2f093"

In [174]:
client_id="PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12"
client_secret="b903c0540ecb05b669cf5b1fe8d640ee42e46a1b340841a00d7e0656d41abaa1"

In [175]:
def get_token(client_id,client_secret):
    data = dict(
                grant_type="client_credentials",
                client_id=client_id,
                client_secret=client_secret,
                scope="api_offresdemploiv2 o2dsoffre application_{}".format(client_id),
            )

    headers = {"content-type": "application/x-www-form-urlencoded"}
    params = dict(realm="/partenaire")
    current_time = datetime.datetime.today()
    r = requests.post(
        url="https://entreprise.pole-emploi.fr/connexion/oauth2/access_token",
        headers=headers,
        data=data,
        params=params,
        timeout=60
            )
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + str(r.json())
            raise HTTPError(complete_message)
        else:
            raise error
    else:
        token = r.json()
        token["expires_at"] = current_time + datetime.timedelta(
            seconds=token["expires_in"]
        )
        
        return token

In [176]:
token=get_token(client_id,client_secret)
token

{'access_token': 'dzAOUrgi4c3xO_2FZpVkv9CrWPM',
 'scope': 'application_PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12 api_offresdemploiv2 o2dsoffre',
 'token_type': 'Bearer',
 'expires_in': 1499,
 'expires_at': datetime.datetime(2024, 1, 9, 21, 9, 52, 156458)}

In [177]:
def get_hearders(token):
    headers = {
            "Authorization": "Bearer {}".format(token["access_token"])
        }
    return headers


In [178]:
# Valeurs constantes

OFFRES_DEMPLOI_V2_BASE = "https://api.emploi-store.fr/partenaire/offresdemploi/v2"
REFERENTIEL_ENDPOINT = "{}/referentiel".format(OFFRES_DEMPLOI_V2_BASE)
SEARCH_ENDPOINT = "{}/offres/search".format(OFFRES_DEMPLOI_V2_BASE)

In [179]:
# 'Réferentiel' available: domaine, appellations (domaines professionnelles ROME), metiers, themes, continents,
#         pays, regions, departements , communes , secteursActivites, naturesContrats,  typesContrats, niveauxFormations,
#         permis, langues

def referentiel(ref):
    
    r = requests.Session().get(
                url="{}/{}".format(REFERENTIEL_ENDPOINT, ref),
                params= dict(realm="/partenaire"),
                headers=get_hearders()
            )

    try:
        r.raise_for_status()
    except Exception as e:
        raise e
    else:
        return r.json()

In [180]:
def search(params=None):
    print('Requête avec les params {}'.format(params))
    r = requests.Session().get(
            url=SEARCH_ENDPOINT,
            params=params,
            headers=get_hearders(token)
        )

    silent_http_errors=False
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + r.json()["message"]
            if silent_http_errors:
                print(complete_message)
            else:
                raise HTTPError(complete_message)
        else:
            if silent_http_errors:
                print(str(error))
            else:
                raise error
    else:
        found_range = re.search(
            pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",
            string=r.headers["Content-Range"],
        ).groupdict()
        out = r.json()
        out.update({"Content-Range": found_range})
        return out

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\barai\AppData\Local\Temp\ipykernel_19800\1347239471.py:26: SyntaxWarning: invalid escape sequence '\d'
  pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",


# Exemple

In [181]:
start_dt = datetime.datetime(2020, 3, 1, 12, 30)
end_dt = datetime.datetime.today()
params = {

    "motsCles": "data",
    'minCreationDate': start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
    'maxCreationDate': end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
}

ex_search=search(params)


Requête avec les params {'motsCles': 'data', 'minCreationDate': '2020-03-01T12:30:00Z', 'maxCreationDate': '2024-01-09T20:44:53Z'}


In [182]:
ex_search

{'resultats': [{'id': '167KSDF',
   'intitule': 'Data Analyst (H/F)',
   'description': "Notre société :\n\nIris Galerie est une société internationale qui propose l'expérience unique de révéler la beauté des yeux grâce au procédé de macrophotographie.\nNous créons à partir de l'iris de nos clients une photographie d'art unique et originale disponible à travers un large panel de formats et finitions. \n\nNous offrons une expérience artistique inoubliable et pleine de surprises, accessible au plus grand nombre.\nIris Galerie dispose d'un réseau de 120 galeries ouvertes en moins de 2 ans dans 17 pays et un projet d'ouvertures de plus de 100 galeries pour l'année 2024 en France et à l'international dans des villes à fort attrait culturel.\n\nIris Galerie poursuit son développement et recherche un Data Analyst H/F pour accompagner sa croissance.\n\nSi vous souhaitez participer à une aventure entrepreneuriale originale et ambitieuse et devenir un des futurs ambassadeurs d'Iris Galerie, rejo

In [183]:
ex_search_results = ex_search.get('resultats', [])  # Récupérer la liste des résultats

# Récupérer les descriptions
descriptions_offres = [offre.get('description', '') for offre in ex_search_results]
descriptions_offres

["Notre société :\n\nIris Galerie est une société internationale qui propose l'expérience unique de révéler la beauté des yeux grâce au procédé de macrophotographie.\nNous créons à partir de l'iris de nos clients une photographie d'art unique et originale disponible à travers un large panel de formats et finitions. \n\nNous offrons une expérience artistique inoubliable et pleine de surprises, accessible au plus grand nombre.\nIris Galerie dispose d'un réseau de 120 galeries ouvertes en moins de 2 ans dans 17 pays et un projet d'ouvertures de plus de 100 galeries pour l'année 2024 en France et à l'international dans des villes à fort attrait culturel.\n\nIris Galerie poursuit son développement et recherche un Data Analyst H/F pour accompagner sa croissance.\n\nSi vous souhaitez participer à une aventure entrepreneuriale originale et ambitieuse et devenir un des futurs ambassadeurs d'Iris Galerie, rejoignez-nous !\n\nVotre mission:\n\nSous la responsabilité du Directeur du Système d'Info

# Traitement de la description

In [184]:
#récupérer la liste des ponctuations
import string
import ssl 
import nltk

ponctuations = list(string.punctuation)
print(ponctuations)


#outil pour procéder à la lemmatisation - attention à charger le cas échéant
#nltk.download()
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

#pour la tokénisation
from nltk.tokenize import word_tokenize

#liste des mots vides
from nltk.corpus import stopwords
mots_vides = stopwords.words("french")
print(mots_vides)

#********************************
#fonction pour nettoyage document (chaîne de caractères)
#le document revient sous la forme d'une liste de tokens
#********************************
def nettoyage_doc(doc_param):
    #passage en minuscule
    doc = doc_param.lower()
    #retrait des ponctuations
    doc = "".join([w for w in list(doc) if not w in ponctuations])
    #transformer le document en liste de termes par tokénisation
    doc = word_tokenize(doc)
    #lematisation de chaque terme
    doc = [lem.lemmatize(terme) for terme in doc]
    #retirer les stopwords
    doc = [w for w in doc if not w in mots_vides]
    #retirer les termes de moins de 3 caractères
    doc = [w for w in doc if len(w)>=1]
    #fin
    return doc

#************************************************************
#fonction pour nettoyage corpus
#attention, optionnellement les documents vides sont éliminés
#************************************************************
def nettoyage_corpus(corpus,vire_vide=True):
    #output
    output = [nettoyage_doc(doc) for doc in corpus if ((len(doc) > 0) or (vire_vide == False))]
    return output


#corpus après pré-traitement, sous forme de listes de liste de tokens
desc = nettoyage_corpus(descriptions_offres)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fus

In [185]:
import sqlite3

In [186]:
conn = sqlite3.connect('job_mining.db')

In [187]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS LieuTravail_Dimension (
    id TEXT PRIMARY KEY,
    libelle TEXT,
    latitude REAL,
    longitude REAL,
    codePostal TEXT,
    commune TEXT
)
''')



In [188]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Entreprise_Dimension (
    id TEXT PRIMARY KEY,
    nom TEXT,
    adaptee INTEGER
)
''')


In [189]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Contrat_Dimension (
    id TEXT PRIMARY KEY,
    typeContrat TEXT,
    typeContratLibelle TEXT,
    natureContrat TEXT,
    dureeTravailLibelle TEXT,
    dureeTravailLibelleConverti TEXT,
    alternance INTEGER
)
''')

In [190]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OrigineOffre_Dimension (
        id TEXT PRIMARY KEY,
        origine TEXT,
        urlOrigine TEXT
    )
''')

In [191]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Qualification_Dimension (
        id TEXT PRIMARY KEY,
        qualification TEXT
    )
''')

In [192]:
# Création de la table de faits (OffresEmploi_Faits)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OffresEmploi_Faits (
        id TEXT PRIMARY KEY,
        dateCreation TEXT,
        dateActualisation TEXT,
        description TEXT,
        nombrePostes INTEGER,
        salaireLibelle TEXT,
        lieuTravailId TEXT REFERENCES LieuTravail_Dimension(id),
        entrepriseId TEXT REFERENCES Entreprise_Dimension(id),
        contratId TEXT REFERENCES Contrat_Dimension(id),
        qualificationId TEXT REFERENCES Qualification_Dimension(id),
        origineOffreId TEXT REFERENCES OrigineOffre_Dimension(id)
    )
''')

In [193]:
# Commit et fermeture de la connexion
conn.commit()
conn.close()

## Chargement table  --- LieuTravail_Dimension

In [194]:
data = {
    'id': [],
    'libelle': [],
    'latitude': [],
    'longitude': [],
    'codePostal': [],
    'commune': []
}

for job_posting in ex_search.get('resultats', []):
    lieu_travail = job_posting.get('lieuTravail', {})
    data['id'].append(lieu_travail.get('libelle', ''))
    data['libelle'].append(lieu_travail.get('libelle', ''))
    data['latitude'].append(lieu_travail.get('latitude', ''))
    data['longitude'].append(lieu_travail.get('longitude', ''))
    data['codePostal'].append(lieu_travail.get('codePostal', ''))
    data['commune'].append(lieu_travail.get('commune', ''))

# Create a DataFrame
df_lieu = pd.DataFrame(data)
df_lieu = df_lieu.drop_duplicates(subset=['libelle'])
df_lieu = df_lieu.reset_index(drop=True)
df_lieu['id'] = df_lieu.index + 1
df_lieu.head()

,id,libelle,latitude,longitude,codePostal,commune
0,1,75 - PARIS 08,48.873635,2.3116,75008,75108
1,2,75 - PARIS 14,48.829706,2.322894,75014,75114
2,3,78 - MONTIGNY LE BRETONNEUX,48.779563,2.033014,78180,78423
3,4,75 - PARIS 09,48.877098,2.337887,75009,75109
4,5,31 - TOULOUSE,43.604082,1.433805,31000,31555


## Chargement de la table --- Entreprise_Dimension

In [195]:
data_entreprises = {
    'id': [],
    'nom': [],
    'adaptee': []
}

for job_posting in ex_search.get('resultats', []):
    entreprise = job_posting.get('entreprise', {})
    
    data_entreprises['id'].append(entreprise.get('nom', ''))
    data_entreprises['nom'].append(entreprise.get('nom', ''))
    data_entreprises['adaptee'].append(entreprise.get('entrepriseAdaptee', ''))

# Create a DataFrame for entreprises
df_entreprises = pd.DataFrame(data_entreprises)
df_entreprises = df_entreprises.drop_duplicates(subset=['nom'])
df_entreprises = df_entreprises.reset_index(drop=True)
df_entreprises['id'] = df_entreprises.index + 1
df_entreprises.head()


,id,nom,adaptee
0,1,IRIS GALERIE,False
1,2,GROUPE HOSPITALIER UNIVERSITAIRE PARIS -,False
2,3,EXPLEO FRANCE,False
3,4,,False
4,5,CS GROUP - FRANCE,False


## Chargement de la table --- Contrat_Dimension

In [196]:
data_contrat = {
    'id': [],
    'typeContrat': [],
    'typeContratLibelle': [],
    'natureContrat': [],
    'dureeTravailLibelle': [],
    'dureeTravailLibelleConverti': [],
    'alternance': []
}

for job_posting in ex_search.get('resultats', []):
    contrat = job_posting
    
    data_contrat['id'].append(contrat.get('id', ''))
    data_contrat['typeContrat'].append(contrat.get('typeContrat', ''))
    data_contrat['typeContratLibelle'].append(contrat.get('typeContratLibelle', ''))
    data_contrat['natureContrat'].append(contrat.get('natureContrat', ''))
    data_contrat['dureeTravailLibelle'].append(contrat.get('dureeTravailLibelle', ''))
    data_contrat['dureeTravailLibelleConverti'].append(contrat.get('dureeTravailLibelleConverti', ''))
    data_contrat['alternance'].append(contrat.get('alternance', ''))

# Create a DataFrame for contrat
df_contrat = pd.DataFrame(data_contrat)
df_contrat['id'] = df_contrat.index + 1
df_contrat.head()


,id,typeContrat,typeContratLibelle,natureContrat,dureeTravailLibelle,dureeTravailLibelleConverti,alternance
0,1,CDI,Contrat à durée indéterminée,Contrat travail,35H Travail en journée,Temps plein,False
1,2,CDD,Contrat à durée déterminée - 6 Mois,Contrat travail,35H Travail en journée,Temps plein,False
2,3,CDI,Contrat à durée indéterminée,Contrat travail,38H30 Travail en journée,Temps plein,False
3,4,CDI,Contrat à durée indéterminée,Contrat travail,35H Travail en journée,Temps plein,False
4,5,CDI,Contrat à durée indéterminée,Contrat travail,36H40 Travail en journée,Temps plein,False


## Chargement de la table --- OrigineOffre_Dimension

In [197]:
data_origine = {
    'id': [],
    'origine': [],
    'urlOrigine': []
}

for job_posting in ex_search.get('resultats', []):
    origine_offre = job_posting.get('origineOffre', {})
    
    data_origine['id'].append(origine_offre.get('id', ''))
    data_origine['origine'].append(origine_offre.get('origine', ''))
    data_origine['urlOrigine'].append(origine_offre.get('urlOrigine', ''))

# Create a DataFrame for origine
df_origine = pd.DataFrame(data_origine)
df_origine['id'] = df_origine.index + 1
df_origine

,id,origine,urlOrigine
0,1,1,https://candidat.pole-emploi.fr/offres/recherc...
1,2,1,https://candidat.pole-emploi.fr/offres/recherc...
2,3,1,https://candidat.pole-emploi.fr/offres/recherc...
3,4,1,https://candidat.pole-emploi.fr/offres/recherc...
4,5,1,https://candidat.pole-emploi.fr/offres/recherc...
...,...,...,...
145,146,1,https://candidat.pole-emploi.fr/offres/recherc...
146,147,1,https://candidat.pole-emploi.fr/offres/recherc...
147,148,1,https://candidat.pole-emploi.fr/offres/recherc...
148,149,1,https://candidat.pole-emploi.fr/offres/recherc...


## Chargement de la table --- Qualification_Dimension

In [198]:
data_qualification = {
    'id': [],
    'qualification': []
}
for job_posting in ex_search.get('resultats', []):
    data_qualification['id'].append(job_posting.get('id', ''))
    data_qualification['qualification'].append(job_posting.get('qualificationLibelle', ''))

# Create a DataFrame for qualification
df_qualification = pd.DataFrame(data_qualification)
df_qualification = df_qualification.drop_duplicates(subset=['qualification'])
df_qualification = df_qualification.reset_index(drop=True)
df_qualification['id'] = df_qualification.index + 1
df_qualification


,id,qualification
0,1,Agent de maîtrise
1,2,Cadre
2,3,Employé qualifié
3,4,Technicien
4,5,Employé non qualifié


## Chargement de la table --- OffresEmploi_Faits

In [199]:
desc[1]

['description',
 'poste',
 'mission',
 'département',
 'rattachement',
 'direction',
 'linnovation',
 'technologique',
 'système',
 'dinformation',
 'ditsi',
 'pôle',
 'innovationrecherche',
 'pôle',
 'assure',
 'suivi',
 'promotion',
 'innovation',
 'technologique',
 'sappuyant',
 'modèles',
 'cocréation',
 'idéation',
 'prototypage',
 'concrétisation',
 'développement',
 'doutils',
 'interne',
 'secteur',
 'participe',
 'nombreux',
 'projets',
 'recherche',
 'clinique',
 'rhu',
 'asphalt',
 'h2020',
 '«',
 'indemand',
 '»',
 'apportant',
 'contribution',
 'technologique',
 'réalisation',
 'digitale',
 'ditsi',
 'a',
 'mission',
 'dassurer',
 'sécurité',
 'bon',
 'fonctionnement',
 'système',
 'dinformation',
 'si',
 'ainsi',
 'développement',
 'cohérence',
 'stratégies',
 'nationales',
 'régionales',
 'plus',
 'précisément',
 'léquipe',
 'direction',
 'linnovation',
 'technologique',
 'système',
 'dinformation',
 'veut',
 'proche',
 'utilisateurs',
 'patientsrésidents',
 'etou',
 'us

In [204]:
import sqlite3
import pandas as pd

# Extracting data for all job postings
data = {
    'id': [],
    'dateCreation': [],
    'dateActualisation': [],
    'description': [],
    'nombrePostes': [],
    'salaireLibelle': [],
    'lieuTravailId': [],
    'entrepriseId': [],
    'contratId': [],
    'qualificationId': [],
    'origineOffreId': []
}

for job_posting in ex_search.get('resultats', []):
    data['id'].append(job_posting.get('id', ''))
    data['dateCreation'].append(job_posting.get('dateCreation', ''))
    data['dateActualisation'].append(job_posting.get('dateActualisation', ''))
    data['nombrePostes'].append(job_posting.get('nombrePostes', ''))
    data['salaireLibelle'].append(job_posting.get('salaire', {}).get('libelle', ''))
    data['lieuTravailId'].append(job_posting.get('lieuTravail', {}).get('libelle', ''))
    data['entrepriseId'].append(job_posting.get('entreprise', {}).get('nom', ''))
    data['contratId'].append(job_posting.get('typeContratLibelle', ''))
    data['qualificationId'].append(job_posting.get('qualificationLibelle', ''))
    data['origineOffreId'].append(job_posting.get('origineOffre', {}).get('origine', ''))


for i in range(0,len(ex_search.get('resultats', []))):
     data['description'].append(desc[i])


# Create a DataFrame
df_offreF = pd.DataFrame(data)

# Convertir la colonne 'description' de liste à chaîne de caractères
df_offreF['description'] = df_offreF['description'].apply(lambda x: ' '.join(x))



In [205]:
df_offreF

,id,dateCreation,dateActualisation,description,nombrePostes,salaireLibelle,lieuTravailId,entrepriseId,contratId,qualificationId,origineOffreId
0,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",75 - PARIS 08,IRIS GALERIE,Contrat à durée indéterminée,Agent de maîtrise,1
1,167KPXR,2024-01-09T15:28:38.000Z,2024-01-09T18:07:09.000Z,description poste mission département rattache...,1,"Mensuel de 2300,00 Euros à 3300,00 Euros sur 1...",75 - PARIS 14,GROUPE HOSPITALIER UNIVERSITAIRE PARIS -,Contrat à durée déterminée - 6 Mois,Cadre,1
2,167KKHH,2024-01-09T14:13:30.000Z,2024-01-09T20:43:09.000Z,sein département digital emerging technology a...,1,"Annuel de 46000,00 Euros à 47000,00 Euros sur ...",78 - MONTIGNY LE BRETONNEUX,EXPLEO FRANCE,Contrat à durée indéterminée,Cadre,1
3,167KFLT,2024-01-09T12:25:42.000Z,2024-01-09T16:32:31.000Z,genius tech recrute lun client data engineer e...,1,"Mensuel de 50000,00 Euros à 60000,00 Euros sur...",75 - PARIS 09,,Contrat à durée indéterminée,Cadre,1
4,167JQQR,2024-01-09T09:52:22.000Z,2024-01-09T09:52:24.000Z,recrutons ingénieur data gouvernance fh rejoin...,2,"Annuel de 38000,00 Euros à 50000,00 Euros sur ...",31 - TOULOUSE,CS GROUP - FRANCE,Contrat à durée indéterminée,Cadre,1
...,...,...,...,...,...,...,...,...,...,...,...
145,166QRFD,2023-12-21T09:13:47.000Z,2024-01-09T09:57:13.000Z,poste 100 cdi mobilité interne détachement mut...,1,"Mensuel de 2237,23 Euros à 2415,67 Euros sur 1...",02 - ST QUENTIN,CENTRE HOSPITALIER GENERAL ST QUENTIN,Contrat à durée indéterminée,Employé qualifié,1
146,166MBGZ,2023-12-18T14:43:43.000Z,2024-01-09T15:37:06.000Z,aussi pensez tout mesurable intéressé solution...,1,"Annuel de 37000,00 Euros sur 12 mois",92 - COURBEVOIE,OTCHEE,Contrat à durée indéterminée,Cadre,1
147,166JQMB,2023-12-15T08:50:55.000Z,2024-01-03T12:40:33.000Z,contexte sensibilité protection données souhai...,1,"Mensuel de 1850,00 Euros à 2240,00 Euros sur 1...",22 - PLERIN,CDGFPT,Contrat à durée déterminée - 12 Mois,Technicien,1
148,166FDCY,2023-12-12T14:09:22.000Z,2024-01-05T15:22:20.000Z,accompagner croissance ladico recherche délégu...,1,"Mensuel de 2100,00 Euros sur 12 mois",60 - BEAUVAIS,Adico,Contrat à durée indéterminée,Agent de maîtrise,1


In [206]:
df_lieu_selection = df_lieu[['id', 'libelle']]
df_entreprises_selection = df_entreprises[['id', 'nom']]
df_contrat_selection = df_contrat[['id', 'typeContratLibelle']]
df_origine_selection = df_origine[['id', 'origine']]
df_qualification_selection = df_qualification[['id', 'qualification']]

#LIEU
df_offreF = pd.merge(df_offreF, df_lieu_selection, how='left', left_on='lieuTravailId', right_on='libelle')
df_offreF = df_offreF.drop(columns=['lieuTravailId','libelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'lieuTravailId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ENTREPRISE
df_offreF = pd.merge(df_offreF, df_entreprises_selection, how='left', left_on='entrepriseId', right_on='nom')
df_offreF = df_offreF.drop(columns=['entrepriseId','nom'])
df_offreF = df_offreF.rename(columns={'id_y': 'entrepriseId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#CONTRAT
df_offreF = pd.merge(df_offreF, df_contrat_selection, how='left', left_on='contratId', right_on='typeContratLibelle')
df_offreF = df_offreF.drop(columns=['contratId','typeContratLibelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'contratId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ORIGINE
df_offreF['origineOffreId'] = df_offreF['origineOffreId'].astype(str)
df_offreF = pd.merge(df_offreF, df_origine_selection, how='left', left_on='origineOffreId', right_on='origine')
df_offreF = df_offreF.drop(columns=['origineOffreId','origine'])
df_offreF = df_offreF.rename(columns={'id_y': 'origineOffreId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#QUALIFICATION
df_offreF = pd.merge(df_offreF, df_qualification_selection, how='left', left_on='qualificationId', right_on='qualification')
df_offreF = df_offreF.drop(columns=['qualificationId','qualification'])
df_offreF = df_offreF.rename(columns={'id_y': 'qualificationId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

# Liste des colonnes dans le nouvel ordre souhaité
nouvel_ordre_colonnes = [
    'id',
    'dateCreation',
    'dateActualisation',
    'description',
    'nombrePostes',
    'salaireLibelle',
    'lieuTravailId',
    'entrepriseId',
    'contratId',
    'qualificationId',
    'origineOffreId'
]
df_offreF = df_offreF[nouvel_ordre_colonnes]

In [207]:
df_offreF.head(10)

,id,dateCreation,dateActualisation,description,nombrePostes,salaireLibelle,lieuTravailId,entrepriseId,contratId,qualificationId,origineOffreId
0,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,1
1,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,2
2,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,3
3,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,4
4,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,5
5,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,6
6,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,7
7,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,8
8,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,9
9,167KSDF,2024-01-09T16:03:25.000Z,2024-01-09T20:33:31.000Z,société iris galerie société internationale pr...,1,"Mensuel de 2000,00 Euros à 3000,00 Euros sur 1...",1,1,1,1,10


## CHARGEMENT DES TABLES

In [208]:
import sqlite3
import pandas as pd
# Connexion à la base de données SQLite
conn = sqlite3.connect('job_mining.db')
###
# Chargement des tables
df_lieu.to_sql('LieuTravail_Dimension', conn, if_exists='replace', index=False)
df_entreprises.to_sql('Entreprise_Dimension', conn, if_exists='replace', index=False)
df_contrat.to_sql('Contrat_Dimension', conn, if_exists='replace', index=False)
df_origine.to_sql('OrigineOffre_Dimension', conn, if_exists='replace', index=False)
df_qualification.to_sql('Qualification_Dimension', conn, if_exists='replace', index=False)
df_offreF.to_sql('OffresEmploi_Faits', conn, if_exists='replace', index=False)

# Fermer la connexion
conn.close()
